In [1]:
import pandas as pd

In [2]:
properati = pd.read_csv('datos/properati-AR-2017-08-01-properties-sell-six_months.csv')
len(properati)

121220

In [3]:
properati.columns

Index([u'id', u'created_on', u'operation', u'property_type', u'place_name',
       u'place_with_parent_names', u'country_name', u'state_name',
       u'geonames_id', u'lat-lon', u'lat', u'lon', u'price', u'currency',
       u'price_aprox_local_currency', u'price_aprox_usd',
       u'surface_total_in_m2', u'surface_covered_in_m2', u'price_usd_per_m2',
       u'price_per_m2', u'floor', u'rooms', u'expenses', u'properati_url',
       u'description', u'title', u'image_thumbnail'],
      dtype='object')

In [4]:
filtrado = properati.drop( ['id','operation','title','description','image_thumbnail','properati_url'], 1)
filtrado.dropna(how='all',inplace=True)
filtrado

,created_on,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,lon,...,currency,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses
0,2017-02-01,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,...,USD,1093959.00,62000.00,55.0,40.0,1127.272727,1550.000000,NaN,NaN,NaN
1,2017-02-01,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,...,USD,2646675.00,150000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-02-01,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,...,USD,1270404.00,72000.00,55.0,55.0,1309.090909,1309.090909,NaN,NaN,NaN
3,2017-02-01,PH,Liniers,|Argentina|Capital Federal|Liniers|,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,...,USD,1676227.50,95000.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-02-01,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,...,USD,1129248.00,64000.00,35.0,35.0,1828.571429,1828.571429,NaN,NaN,NaN
5,2017-02-01,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|,Argentina,Entre Ríos,3433657.0,"-33.0140714,-58.519828",-33.014071,-58.519828,...,NaN,NaN,NaN,53.0,NaN,NaN,NaN,NaN,NaN,NaN
6,2017-02-01,PH,Munro,|Argentina|Bs.As. G.B.A. Zona Norte|Vicente Ló...,Argentina,Bs.As. G.B.A. Zona Norte,3430511.0,"-34.5329567,-58.5217825",-34.532957,-58.521782,...,USD,2293785.00,130000.00,106.0,78.0,1226.415094,1666.666667,NaN,NaN,NaN
7,2017-02-01,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,USD,2434941.00,138000.00,45.0,40.0,3066.666667,3450.000000,NaN,NaN,NaN
8,2017-02-01,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,...,USD,3440677.50,195000.00,65.0,60.0,3000.000000,3250.000000,NaN,NaN,NaN
9,2017-02-01,house,Rosario,|Argentina|Santa Fe|Rosario|,Argentina,Santa Fe,3838574.0,"-32.942031,-60.7259192",-32.942031,-60.725919,...,ARS,455201.45,25798.49,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
#separo place_with_parent_names
separado= filtrado['place_with_parent_names'].apply(lambda x: pd.Series(x.split('|')))
# estaria bueno que las columnas tengan el nombre que corresponde y la 0 no tenga nombre
separado.columns = ['a','Pais', 'Distrito','Localidad','Ciudad','Barrio','aborrar'] #mejorar
separado = separado.drop(['a','aborrar'],1) # mejorar
separado.head()

,Pais,Distrito,Localidad,Ciudad,Barrio
0,Argentina,Capital Federal,Mataderos,,NaN
1,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,,NaN
2,Argentina,Capital Federal,Mataderos,,NaN
3,Argentina,Capital Federal,Liniers,,NaN
4,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Centro,


In [17]:
unido = pd.concat([filtrado, separado], axis=1)
unido.drop('place_with_parent_names',1)

,created_on,property_type,place_name,country_name,state_name,geonames_id,lat-lon,lat,lon,price,...,price_usd_per_m2,price_per_m2,floor,rooms,expenses,Pais,Distrito,Localidad,Ciudad,Barrio
0,2017-02-01,PH,Mataderos,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,-58.508839,62000.0,...,1127.272727,1550.000000,NaN,NaN,NaN,Argentina,Capital Federal,Mataderos,,NaN
1,2017-02-01,apartment,La Plata,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,-57.964330,150000.0,...,NaN,NaN,NaN,NaN,NaN,Argentina,Bs.As. G.B.A. Zona Sur,La Plata,,NaN
2,2017-02-01,apartment,Mataderos,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,-58.522982,72000.0,...,1309.090909,1309.090909,NaN,NaN,NaN,Argentina,Capital Federal,Mataderos,,NaN
3,2017-02-01,PH,Liniers,Argentina,Capital Federal,3431333.0,"-34.6477969,-58.5164244",-34.647797,-58.516424,95000.0,...,NaN,NaN,NaN,NaN,NaN,Argentina,Capital Federal,Liniers,,NaN
4,2017-02-01,apartment,Centro,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,-57.549447,64000.0,...,1828.571429,1828.571429,NaN,NaN,NaN,Argentina,Buenos Aires Costa Atlántica,Mar del Plata,Centro,
5,2017-02-01,house,Gualeguaychú,Argentina,Entre Ríos,3433657.0,"-33.0140714,-58.519828",-33.014071,-58.519828,NaN,...,NaN,NaN,NaN,NaN,NaN,Argentina,Entre Ríos,Gualeguaychú,,NaN
6,2017-02-01,PH,Munro,Argentina,Bs.As. G.B.A. Zona Norte,3430511.0,"-34.5329567,-58.5217825",-34.532957,-58.521782,130000.0,...,1226.415094,1666.666667,NaN,NaN,NaN,Argentina,Bs.As. G.B.A. Zona Norte,Vicente López,Munro,
7,2017-02-01,apartment,Belgrano,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,138000.0,...,3066.666667,3450.000000,NaN,NaN,NaN,Argentina,Capital Federal,Belgrano,,NaN
8,2017-02-01,apartment,Belgrano,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,-58.443362,195000.0,...,3000.000000,3250.000000,NaN,NaN,NaN,Argentina,Capital Federal,Belgrano,,NaN
9,2017-02-01,house,Rosario,Argentina,Santa Fe,3838574.0,"-32.942031,-60.7259192",-32.942031,-60.725919,460000.0,...,NaN,NaN,NaN,NaN,NaN,Argentina,Santa Fe,Rosario,,NaN
